In [1]:
import torch
import torch.nn as nn
from torch.nn.utils import weight_norm

In [2]:
class CDIL_Block(nn.Module):
    def __init__(self, c_in, c_out, ks, pad, dil):
        super(CDIL_Block, self).__init__()
        self.conv = weight_norm(nn.Conv1d(c_in, c_out, ks, padding=pad, dilation=dil, padding_mode='circular'))
        self.conv.weight.data.normal_(0, 0.01)
        self.conv.bias.data.normal_(0, 0.01)

        self.res = nn.Conv1d(c_in, c_out, kernel_size=(1,)) if c_in != c_out else None
        if self.res is not None:
            self.res.weight.data.normal_(0, 0.01)
            self.res.bias.data.normal_(0, 0.01)

        self.nonlinear = nn.ReLU()

    def forward(self, x):
        out = self.conv(x)
        res = x if self.res is None else self.res(x)
        return self.nonlinear(out) + res

In [3]:
class CDIL_ConvPart(nn.Module):
    def __init__(self, dim_in, hidden_channels, ks=3):
        super(CDIL_ConvPart, self).__init__()
        layers = []
        num_layer = len(hidden_channels)
        for i in range(num_layer):
            this_in = dim_in if i == 0 else hidden_channels[i - 1]
            this_out = hidden_channels[i]
            this_dilation = 2 ** i
            this_padding = int(this_dilation * (ks - 1) / 2)
            layers += [CDIL_Block(this_in, this_out, ks, this_padding, this_dilation)]
        self.conv_net = nn.Sequential(*layers)

    def forward(self, x):
        return self.conv_net(x)

# 1. cdil-cnn convolutional part outputs the same size(length) as the input sequence

In [4]:
SEQ_LENGTH = 100  # remain unchanged
INPUT_DIM = 10
BATCH = 32
x = torch.rand(BATCH, INPUT_DIM, SEQ_LENGTH)
x.shape

torch.Size([32, 10, 100])

In [5]:
HIDDEN_CHANNEL = 50
LAYER = 4
cdil_conv_part1 = CDIL_ConvPart(INPUT_DIM, [HIDDEN_CHANNEL] * LAYER)
y_conv = cdil_conv_part1(x)
y_conv.shape

torch.Size([32, 50, 100])

In [6]:
cdil_conv_part2 = CDIL_ConvPart(INPUT_DIM, [20, 30, 40])
y_conv = cdil_conv_part2(x)
y_conv.shape

torch.Size([32, 40, 100])

# 2. cdil-cnn model (classifier)

In [7]:
class CDIL_CNN(nn.Module):
    def __init__(self, input_size, output_size, num_channels, kernel_size=3, use_embed=False, char_vocab=None):
        super(CDIL_CNN, self).__init__()

        self.use_embed = use_embed
        if self.use_embed:
            self.embedding = nn.Embedding(char_vocab, input_size)

        self.conv = CDIL_ConvPart(input_size, num_channels, kernel_size)
        self.classifier = nn.Linear(num_channels[-1], output_size)

    def forward(self, x):
        if self.use_embed:
            x = self.embedding(x)
            x = x.permute(0, 2, 1).to(dtype=torch.float)
        y_conv = self.conv(x)  # x, y: num, channel(dim), length
        y = self.classifier(torch.mean(y_conv, dim=2))
        return y

## using input sequences without embedding

In [8]:
SEQ_LENGTH = 200
INPUT_DIM = 3
OUTPUT_CLASS = 11
BATCH = 64
HIDDEN_CHANNEL = 20
LAYER = 3
x = torch.rand(BATCH, INPUT_DIM, SEQ_LENGTH)
x.shape

torch.Size([64, 3, 200])

In [9]:
cdil_model_noembed = CDIL_CNN(INPUT_DIM, OUTPUT_CLASS, [HIDDEN_CHANNEL] * LAYER)
y = cdil_model_noembed(x)
y.shape

torch.Size([64, 11])

## using input sequences with embedding

In [10]:
USE_EMBED = True
CHAR_VOCAB = 5

SEQ_LENGTH = 300
EMBED_DIM = 4
OUTPUT_CLASS = 13
BATCH = 16
HIDDEN_CHANNEL = 30
LAYER = 4

In [11]:
x = torch.randint(CHAR_VOCAB, (BATCH, SEQ_LENGTH))
x.shape

torch.Size([16, 300])

In [12]:
cdil_model_embed = CDIL_CNN(EMBED_DIM, OUTPUT_CLASS, [HIDDEN_CHANNEL] * LAYER, use_embed=USE_EMBED, char_vocab=CHAR_VOCAB)
y = cdil_model_embed(x)
y.shape

torch.Size([16, 13])